# Introduction to SQL for Excel Users – Part 19: More CASE WHEN

[Original post](https://www.daveondata.com/blog/introduction-to-sql-for-excel-users-part-19-more-case-when/)

## The Scenario

NOTE – The Execl and SQL used in this post is not optimal by design. This post is about communicating the concepts. In the next post I will demonstrate the concepts using more optimal SQL.

Let’s say you are the sales manager for AdventureWorks and you would like to look at the effectiveness of the sales reps.

In particular, you are interesed in knowing how success in the first 90 days might be related to long-term sales success.

In this post I will cover the basics of how to use the mighty CASE WHEN and GROUP BY to facilitate this analysis.

## Setting Up the Data in SQL

To conduct the analysis in SQL, I need a query to get the data into a format that matches the first Excel table:

In [1]:
SELECT CONCAT(E.LastName, ', ', E.FirstName) AS SalesRep 
      ,E.HireDate
      ,FRS.SalesOrderNumber
      ,CAST(FRS.OrderDate AS Date) AS OrderDate
      ,SUM(FRS.SalesAmount) AS SalesAmount
FROM FactResellerSales FRS 
    INNER JOIN DimEmployee E ON (FRS.EmployeeKey = E.EmployeeKey)
GROUP BY E.FirstName, E.LastName, E.HireDate, FRS.SalesOrderNumber, FRS.OrderDate
ORDER BY E.LastName, E.FirstName, FRS.SalesOrderNumber

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'FactResellerSales'.

Total execution time: 00:00:00.003

## Aggregating the Data in SQL

Using SQL we can combine the Excel steps ☝ in a single query.

First, I will move the SQL code into a CTE:

In [ ]:
WITH SalesRepData AS
(
    SELECT CONCAT(E.LastName, ', ', E.FirstName) AS SalesRep
          ,E.HireDate
          ,FRS.SalesOrderNumber
          ,CAST(FRS.OrderDate AS Date) AS OrderDate
          ,SUM(FRS.SalesAmount) AS SalesAmount
     FROM FactResellerSales FRS 
         INNER JOIN DimEmployee E ON (FRS.EmployeeKey = E.EmployeeKey)
     GROUP BY E.FirstName, E.LastName, E.HireDate, FRS.SalesOrderNumber, FRS.OrderDate
)

Next, I will craft the following query that leverages the CTE. 

 The use of CASE WHEN ☝ conceptually mimics what was done in the first Excel table.

In [ ]:
WITH SalesRepData AS
(
    SELECT CONCAT(E.LastName, ', ', E.FirstName) AS SalesRep
          ,E.HireDate
          ,FRS.SalesOrderNumber
          ,CAST(FRS.OrderDate AS Date) AS OrderDate
          ,SUM(FRS.SalesAmount) AS SalesAmount
    FROM FactResellerSales FRS 
        INNER JOIN DimEmployee E ON (FRS.EmployeeKey = E.EmployeeKey)
    GROUP BY E.FirstName, E.LastName, E.HireDate, FRS.SalesOrderNumber, FRS.OrderDate
)
SELECT SRD.SalesRep
      ,SRD.HireDate
      ,SRD.SalesOrderNumber
      ,SRD.OrderDate
      ,SRD.SalesAmount
      ,CASE WHEN DATEDIFF(DAY, SRD.HireDate, SRD.OrderDate) <= 89 THEN 1 ELSE 0 END AS SaleMadeFirst90Days
      ,CASE WHEN DATEDIFF(DAY, SRD.HireDate, SRD.OrderDate) <= 89 THEN SRD.SalesAmount ELSE 0.0 END AS SalesAmountFirst90Days
FROM SalesRepData SRD
ORDER BY SRD.SalesRep, SRD.SalesOrderNumber;


With this SQL in place, it is trivial to mimic the Excel pivot table ☝.

All I need to do is add the mighty GROUP BY and use the SUM function:

In [ ]:
WITH SalesRepData AS
(
     SELECT CONCAT(E.LastName, ', ', E.FirstName) AS SalesRep 
           ,E.HireDate
           ,FRS.SalesOrderNumber
           ,CAST(FRS.OrderDate AS Date) AS OrderDate
           ,SUM(FRS.SalesAmount) AS SalesAmount
     FROM FactResellerSales FRS 
         INNER JOIN DimEmployee E ON (FRS.EmployeeKey = E.EmployeeKey)
     GROUP BY E.FirstName, E.LastName, E.HireDate, FRS.SalesOrderNumber, FRS.OrderDate
)
SELECT SRD.SalesRep
      ,SUM(CASE WHEN DATEDIFF(DAY, SRD.HireDate, SRD.OrderDate) <= 89 THEN 1 ELSE 0 END) AS TotalSalesMadeFirst90Days
      ,SUM(CASE WHEN DATEDIFF(DAY, SRD.HireDate, SRD.OrderDate) <= 89 THEN SRD.SalesAmount ELSE 0.0 END) AS TotalSalesAmountFirst90Days
FROM SalesRepData SRD
GROUP BY SRD.SalesRep
ORDER BY SRD.SalesRep

The following SQL snippet is where the ✨ happens:

```
SUM(CASE WHEN DATEDIFF(DAY, SRD.HireDate, SRD.OrderDate) <= 89 THEN 1 ELSE 0 END) AS TotalSalesMadeFirst90Days
```

Conceptually this SQL code operates exactly like the Excel pivot table’s SUM functionality.

You may be thinking, “Dave, big deal. Seems to me that the Excel pivot table works just fine.”

Yes, this is true when your data is small.

However, with these SQL skills you are no longer limited to Excel’s constraints.

Now you can work with tens of millions of rows of data.

Heck, using big data technologies you could work with billions of rows of data! 😲

Pretty sweet, no? 😉

## The Learning Arc

The next post will continue coverage of CASE WHEN by expanding upon this analysis with more optimal SQL.

With the combination of CASE WHEN and GROUP BY your SQL skills have now achieved a new level of power with working with data at scale.

Stay healthy and happy data sleuthing!